In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [19]:
#  Store Part I results into DataFrame

In [20]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df

Unnamed: 0          City      Lat       Lng  Max Temp  Humidity  \
0             0      kirakira -10.4544  161.9205     26.56        89   
1             1       fortuna  40.5982 -124.1573     12.78        72   
2             2       rikitea -23.1203 -134.9692     26.72        81   
3             3  puerto ayora  -0.7393  -90.3518     26.11        92   
4             4     toamasina -18.1667   49.3833     24.00        94   
..          ...           ...      ...       ...       ...       ...   
544         544       diapaga  12.0708    1.7889     26.07        17   
545         545     morondava -20.2833   44.2833     25.38        92   
546         546      junagarh  19.8667   82.9333     18.55        34   
547         547      espanola  35.9911 -106.0806     13.33        10   
548         548    san marcos  33.1434 -117.1661     12.22        87   

     Cloudiness  Wind Speed Country        Date  
0           100        5.80      SB  1614817398  
1            35        0.45      US  1614817424  
2            49        4.00      PF  1614817303  
3            22        2.03      EC  1614817314  
4            40        1.54      MG  1614817754  
..          ...         ...     ...         ...  
544           0        2.85      BF  1614817871  
545          83        5.01      MG  1614817391  
546          19        0.85      IN  1614817871  
547           1        5.14      US  1614817872  
548          90        1.54      US  1614817872  

[549 rows x 10 columns]

In [25]:
gmaps.configure(api_key=g_key)
locations= weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

In [26]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
max_intensity

100

In [28]:
# Create new DataFrame fitting weather criteria

# Narrow down the cities to fit weather conditions.Drop any rows will null values.

In [30]:
narrowed_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

Empty DataFrame
Columns: [Unnamed: 0, City, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed, Country, Date]
Index: []

In [29]:
# Store into variable named hotel_df
# Add a "Hotel Name" column to the DataFrame
# Set parameters to search for hotels with 5000 meters
# Hit the Google Places API for each city's coordinates
# Store the first Hotel result into the DataFrame
# Plot markers on top of the heatmap

In [31]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

Empty DataFrame
Columns: [City, Country, Lat, Lng, Hotel Name]
Index: []

In [32]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [33]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

-------End of Search-------


In [34]:
hotel_df

Empty DataFrame
Columns: [City, Country, Lat, Lng, Hotel Name]
Index: []

In [ ]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]